# Set Up Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose, Reshape, Dense
from keras import Model
import numpy as np
from tqdm import tqdm
print(f"Using Tensorflow v{tf.__version__}")

Using Tensorflow v2.10.0


# Encoder

In [2]:
encoder = Sequential(name="Encoder")
encoder.add(Conv2D(16, kernel_size=(4,1), strides=(1,2), padding='same', activation='relu', input_shape=(1,256,1), name="Encoder_Conv2D_1"))
encoder.add(Conv2D(32, kernel_size=(4,1), strides=(1,2), padding='same', activation='relu', name="Encoder_Conv2D_2"))
encoder.add(Reshape((1,-1), name="Encoder_Reshape"))
encoder.add(Dense(16, name="Encoder_Dense_1"))
encoder.add(Dense(16, name="Encoder_Dense_2"))

encoder.build()
encoder.summary()

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Encoder_Conv2D_1 (Conv2D)   (None, 1, 128, 16)        80        
                                                                 
 Encoder_Conv2D_2 (Conv2D)   (None, 1, 64, 32)         2080      
                                                                 
 Encoder_Reshape (Reshape)   (None, 1, 2048)           0         
                                                                 
 Encoder_Dense_1 (Dense)     (None, 1, 16)             32784     
                                                                 
 Encoder_Dense_2 (Dense)     (None, 1, 16)             272       
                                                                 
Total params: 35,216
Trainable params: 35,216
Non-trainable params: 0
____________________________________________

2023-02-21 18:05:15.284064: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 18:05:15.284196: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Decoder

In [3]:
decoder = tf.keras.models.Sequential(name="Decoder")
decoder.add(tf.keras.layers.Dense(128, input_shape=(1,16,1), name="Decoder_Dense"))
decoder.add(tf.keras.layers.Reshape((1,64,-1), name="Decoder_Reshape"))
decoder.add(tf.keras.layers.Conv2DTranspose(16, kernel_size=(4,1),strides=(1,2), padding='same', activation='relu', name="Decoder_Conv2D_T_1"))
decoder.add(tf.keras.layers.Conv2DTranspose(1, kernel_size=(4,1),strides=(1,2), padding='same', activation='sigmoid', name="Decoder_Conv2D_T_2"))

decoder.build()
decoder.summary()

Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Decoder_Dense (Dense)       (None, 1, 16, 128)        256       
                                                                 
 Decoder_Reshape (Reshape)   (None, 1, 64, 32)         0         
                                                                 
 Decoder_Conv2D_T_1 (Conv2DT  (None, 1, 128, 16)       2064      
 ranspose)                                                       
                                                                 
 Decoder_Conv2D_T_2 (Conv2DT  (None, 1, 256, 1)        65        
 ranspose)                                                       
                                                                 
Total params: 2,385
Trainable params: 2,385
Non-trainable params: 0
_________________________________________________________________


# Autoencoder

In [4]:
class Autoencoder(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.recon_loss = []
        self.encoder = encoder
        self.decoder = decoder
        self.recon_loss_tracker = keras.metrics.Mean(name="recon_loss")
    
    @property
    def metrics(self):
        return [
            self.recon_loss_tracker
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            recon_results = self.decoder(self.encoder(data))
            recon_loss = tf.reduce_mean(
                tf.reduce_mean(
                    keras.losses.binary_crossentropy(data, recon_results)
                )
            )
        grads = tape.gradient(recon_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.recon_loss_tracker.update_state(recon_loss)
        self.recon_loss.append(recon_loss)

        return {
            "recon_loss": self.recon_loss_tracker.result()
        }

In [5]:
def normalize(tensor):
    tensor_minusmin = tensor - tensor.min()
    tensor_maxminusmin = tensor.max() - tensor.min()
    return tensor_minusmin / tensor_maxminusmin

def get_waves(file_name):
    # Read the raw audio from the .WAV file
    raw_audio = tf.io.read_file(filename=file_name)
    # Convert the raw audio to a waveform
    wave_bank, sample_rate = tf.audio.decode_wav(raw_audio)
    # Display the wavebank and sample_rate
    # display(wave_bank, sample_rate)
    wave_size = 256
    num_waves = 0
    waves = []
    wave_forms = np.transpose(wave_bank.numpy())
    for i in range(64):
        wave_form = wave_forms[0, i*wave_size:i*wave_size+wave_size]
        if wave_form.max() != wave_form.min():
            wave_form = normalize(wave_form)
            waves.append(wave_form)
            num_waves += 1    
    waves = tf.stack(waves)
    return waves

In [6]:
waves = get_waves("./audio_data/ENVELO01.WAV")
display(waves.shape)
waves = tf.expand_dims(waves,1)
waves = tf.expand_dims(waves,1)
waves = tf.expand_dims(waves,-1)
display(waves.shape)

TensorShape([64, 256])

TensorShape([64, 1, 1, 256, 1])

In [7]:
autoencoder = Autoencoder(encoder=encoder, decoder=decoder)
autoencoder.compile(optimizer=keras.optimizers.Adam())

In [8]:
a = decoder(encoder(waves[0]))

In [9]:
# display(a[:,:,0], a[:,:,:-1],a[:,:,-1] - a[:,:,0:])
display(tf.reduce_sum(tf.square(a[:,:,-1] - a[:,:,0:])))

<tf.Tensor: shape=(), dtype=float32, numpy=8.655622e-06>

In [12]:
autoencoder.trainable_weights

[<tf.Variable 'Encoder_Conv2D_1/kernel:0' shape=(4, 1, 1, 16) dtype=float32, numpy=
 array([[[[ 0.0429101 ,  0.08739763,  0.29320073, -0.06408255,
           -0.22795498, -0.26952788,  0.22087318,  0.17738256,
           -0.25935185,  0.23303705, -0.20234317,  0.15313187,
            0.2093358 ,  0.2714665 , -0.2339984 ,  0.0649783 ]]],
 
 
        [[[ 0.16304404,  0.28118467,  0.2649734 ,  0.25992548,
           -0.296829  ,  0.1062825 , -0.17384061, -0.03438255,
           -0.24900527, -0.02949646,  0.27781576, -0.14366397,
            0.1729838 , -0.1798254 , -0.28598285, -0.01399881]]],
 
 
        [[[ 0.2829833 ,  0.26706702, -0.15621145, -0.07906011,
           -0.2100484 , -0.24481374,  0.15595353,  0.29282504,
           -0.22057593,  0.06302315,  0.2489245 , -0.2720341 ,
            0.25290424, -0.14566763,  0.26314104,  0.02067503]]],
 
 
        [[[ 0.1730887 ,  0.19694775,  0.29282832,  0.25823295,
            0.223562  ,  0.20044687, -0.11983778,  0.07090366,
            0

In [10]:
# num_epochs = 50

# for j in tqdm(range(num_epochs)):
#     for i in waves:
#         recon_loss = autoencoder.train_step(i)

In [11]:
autoencoder.encoder()

ValueError: The first argument to `Layer.call` must always be passed.